In [1]:
import os
from enum import StrEnum, auto

import requests
from dotenv import load_dotenv
from oauthlib.oauth2 import BackendApplicationClient
from pydantic import BaseModel, ValidationError
from requests_oauthlib import OAuth2Session

load_dotenv()

True

In [2]:
class BlizzardRegion(StrEnum):
    us = auto()
    eu = auto()
    kr = auto()
    tw = auto()


class AuthToken(BaseModel):
    access_token: str
    token_type: str
    expires_in: int
    sub: str
    expires_at: float


class Region(BaseModel):
    id: int
    name: str


class RealmType(BaseModel):
    name: str
    type: str


class RealmStatus(BaseModel):
    name: str
    type: str


class RealmPopulation(BaseModel):
    name: str
    type: str


class Realm(BaseModel):
    id: int
    slug: str
    name: str
    timezone: str
    locale: str
    is_tournament: bool
    category: str
    type: RealmType
    region: Region


class ConnectedRealm(BaseModel):
    id: int
    href: str
    status: RealmStatus
    population: RealmPopulation
    has_queue: bool
    realms: list[Realm]

In [3]:
def get_blizz_auth_token() -> AuthToken:
    BLIZZ_CLIENT = os.getenv("BLIZZ_CLIENT_ID")
    BLIZZ_SECRET = os.getenv("BLIZZ_CLIENT_SECRET")
    auth_url = 'https://oauth.battle.net/token'
    client = BackendApplicationClient(client_id=BLIZZ_CLIENT)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(token_url=auth_url, client_id=BLIZZ_CLIENT, client_secret=BLIZZ_SECRET)

    try:
        blizz_token: AuthToken = AuthToken.parse_obj(token)
    except ValidationError as e:
        print(e)
    
    return blizz_token


In [4]:
token = get_blizz_auth_token()

SSLError: HTTPSConnectionPool(host='oauth.battle.net', port=443): Max retries exceeded with url: /token (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))

In [4]:
url = f'https://us.api.blizzard.com/data/wow/search/connected-realm?namespace=dynamic-us&orderby=id&_page=1&access_token={token.access_token}'
connected_realms_res = requests.get(url)

realms_dict = connected_realms_res.json()

connected_realms = []
for rc in realms_dict['results']:
    realms = []
    for r in rc['data']['realms']:
        realm: Realm = Realm(
            id=r['id'],
            slug=r['slug'],
            name=r['name']['en_US'],
            timezone=r['timezone'],
            locale=r['locale'],
            is_tournament=r['is_tournament'],
            category=r['category']['en_US'],
            type=RealmType(
                name=r['type']['name']['en_US'],
                type=r['type']['type']
            ),
            region=Region(
                id=r['region']['id'],
                name=r['region']['name']['en_US']
            )
        )
        realms.append(realm)
        
    connected_realm: ConnectedRealm = ConnectedRealm(
        id=rc['data']['id'],
        href=rc['key']['href'],
        has_queue=rc['data']['has_queue'],
        status=RealmStatus(
            name=rc['data']['status']['name']['en_US'],
            type=rc['data']['status']['type']
        ),
        population=RealmPopulation(
            name=rc['data']['population']['name']['en_US'],
            type=rc['data']['population']['type'],
        ),
        realms=realms
    )
    connected_realms.append(connected_realm)


In [5]:
def get_realm_leaderboards(realm_id: int, access_token: str):
    url = f'https://us.api.blizzard.com/data/wow/connected-realm/{realm_id}/mythic-leaderboard/index?namespace=dynamic-us&locale=en_US&access_token={access_token}'

In [11]:
temple = 'https://us.api.blizzard.com/data/wow/connected-realm/5/mythic-leaderboard/2/period/894?namespace=dynamic-us&access_token=USx1btfLiJlNyjkblF1TQey89PGQNaXIzR'


83